In [1]:
import pandas as pd
import networkx as nx

In [ ]:
df_authors = pd.read_csv('/mnt/hdd01/Cleantech Network Analysis/df_authors.csv', dtype={'author_id': str, 'oaid': str, 'appln_id': str})
df_authors = df_authors[['appln_id', 'oaid', 'author_id']]
df_rel_pcs = pd.read_csv('/mnt/hdd01/Cleantech Network Analysis/df_rel_pcs_patstat.csv')
df_oaid_cleantech_dtm = pd.read_csv('/mnt/hdd01/Cleantech Network Analysis/df_oaid_cleantech_dtm.csv')
df_patstat_cleantech_dtm = pd.read_csv('/mnt/hdd01/Cleantech Network Analysis/df_patstat_cleantech_dtm.csv')
df_cleantech_keyword = pd.read_json("/home/thiesen/Documents/Cleantech_Concepts/cleantech_keywords_similarity_015_co_occurrence_025_claim_fulltext.json")

# Add general indices too all nodes and edges in the dataframes

In [ ]:
asdf

# Build Graph Model

In [7]:
G = nx.Graph()

In [ ]:
for node in df_authors['author_id']:
    G.add_node(node, node_type='author')

for node in df_patstat_cleantech_dtm['appln_id']:
    G.add_node(node, node_type='patent')

for node in df_oaid_cleantech_dtm['oaid']:
    G.add_node(node, node_type='paper')

for node in df_cleantech_keyword['keyword_yake_lemma']:
    G.add_node(node, node_type='keyword')

In [ ]:
G.add_edges_from(df_rel_pcs[['appln_id', 'oaid']].values)
G.add_edges_from(df_authors[['appln_id', 'author_id']].values)
G.add_edges_from(df_authors[['oaid', 'author_id']].values)
G.add_edges_from(df_oaid_cleantech_dtm[['oaid', 'keyword_yake_lemma']].values)
G.add_edges_from(df_patstat_cleantech_dtm[['appln_id', 'keyword_yake_lemma']].values)

# Analyze Centrality Measures

In [ ]:
degree_centrality = nx.degree_centrality(G)

In [ ]:
betweenness_centrality = nx.betweenness_centrality(G)

In [ ]:
eigenvector_centrality = nx.eigenvector_centrality(G)